In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
genres = ['Action', 'Drama', 'Thriller', 'Crime', 'Adventure', 'Comedy', 'Sci-Fi', 'Romance', 'Fantasy', 'Horror', 'War', 'Mystery', 'History', 'Animation', 'Western', 'Family', 'Sport', 'Biography', 'Musical', 'Music', 'Film-Noir', 'Reality-TV', 'Game-Show', 'News', 'Adult', 'Talk-Show']
lowercase_genres = [g.lower() for g in genres]
lowercase_genres

['action',
 'drama',
 'thriller',
 'crime',
 'adventure',
 'comedy',
 'sci-fi',
 'romance',
 'fantasy',
 'horror',
 'war',
 'mystery',
 'history',
 'animation',
 'western',
 'family',
 'sport',
 'biography',
 'musical',
 'music',
 'film-noir',
 'reality-tv',
 'game-show',
 'news',
 'adult',
 'talk-show']

In [3]:
from tqdm import tqdm

data = {}

for genre in tqdm(lowercase_genres):
    for num in range(1,1000,50):
        url = "https://www.imdb.com/search/title/?genres="+genre+"&start="+str(num)+"&ref_=adv_nxt"
#         print(url)

        html = requests.get(url)
#         print('Response:', html)
        soup = BeautifulSoup(html.text, 'html.parser')
        movie_div = soup.find_all('div', class_='lister-item mode-advanced')

        for movie in movie_div:
            if movie.find('h3', class_='lister-item-header').find('a') is not None:
                title_element = movie.find('h3', class_='lister-item-header').find('a')
                title = title_element.text.strip() if title_element else None
            else:
                title = None

            if movie.find('img', class_='loadlate') is not None:
                image_element = movie.find('img', class_='loadlate')
                image_url = image_element['src'] if image_element else None
            else:
                image_url = None

            if movie.find('span', class_='lister-item-year text-muted unbold') is not None:
                year_element = movie.find('span', class_='lister-item-year text-muted unbold')
                year = year_element.text.strip('()') if year_element else None
            else:
                year = None

            if movie.find('span', class_='certificate') is not None:
                certificate_element = movie.find('span', class_='certificate')
                certificate = certificate_element.text if certificate_element else None
            else:
                certificate = None

            if movie.find_all('p', class_='text-muted') is not None:
                description_element = movie.find_all('p', class_='text-muted')
                description = description_element[1].text.strip() if len(description_element) >= 2 else None
            else:
                description = None

            if movie.find('div', class_='ratings-bar') is not None:
                rating_element = movie.find('div', class_='ratings-bar').find('strong')
                rating = rating_element.text if rating_element else None
            else:
                rating = None

            if movie.find('p', class_='').find_all('a'):
                director_element = movie.find('p', class_='').find_all('a')
                director = director_element[0].text if director_element else None
                stars = [star.text for star in director_element[1:]] if director_element else None
            else: 
                director = None
                stars = None

            votes_element = movie.find('p', class_='sort-num_votes-visible')
            if votes_element is not None and votes_element.find('span', attrs={'name': 'nv'}) is not None:
                votes = votes_element.find('span', attrs={'name': 'nv'}).text
            else:
                votes = None

            data[title] = {
                'Title': title,
                'Image_URL': image_url,
                'Year': year,
                'Genre': genre,
                'Certificate': certificate,
                'Description': description,
                'Rating': rating,
                'Director': director,
                'Stars': stars,
                'Votes': votes
            }

#             print('Title:', title)
#             print('Year:', year)
#             print('Image URL:', image_url)
#             print('Certificate:', certificate)
#             print('Description:', description)
#             print('Rating:', rating)
#             print('Director:', director)
#             print('Stars:', stars)
#             print('Votes:', votes)
#             print('---')
        
    


100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [20:38<00:00, 47.62s/it]


In [4]:
data

{'Indiana Jones and the Dial of Destiny': {'Title': 'Indiana Jones and the Dial of Destiny',
  'Image_URL': 'https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png',
  'Year': '2023',
  'Genre': 'adventure',
  'Certificate': 'PG-13',
  'Description': 'Archaeologist Indiana Jones races against time to retrieve a legendary artifact that can change the course of history.',
  'Rating': '6.9',
  'Director': 'James Mangold',
  'Stars': ['Harrison Ford',
   'Phoebe Waller-Bridge',
   'Antonio Banderas',
   'Karen Allen'],
  'Votes': '55,758'},
 'The Witcher': {'Title': 'The Witcher',
  'Image_URL': 'https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png',
  'Year': '2019– ',
  'Genre': 'mystery',
  'Certificate': None,
  'Description': 'Geralt of Rivia, a solitary monster hunter, struggles to find his place in a world where people often prove more wicked than beasts.',
  'Rating': '8.1',
  'Director': 'Henry Cavill',
  'Stars': ['Freya Allan', 'Anya Chalotra', 'Eamon Farren'],
  '

In [5]:
import pandas as pd

df = pd.DataFrame.from_dict(data, orient='index')
df = df[['Title', 'Image_URL', 'Year', 'Genre', 'Certificate', 'Description', 'Rating', 'Director', 'Stars', 'Votes']]
df

,Title,Image_URL,Year,Genre,Certificate,Description,Rating,Director,Stars,Votes
Indiana Jones and the Dial of Destiny,Indiana Jones and the Dial of Destiny,https://m.media-amazon.com/images/S/sash/4Fyxw...,2023,adventure,PG-13,Archaeologist Indiana Jones races against time...,6.9,James Mangold,"[Harrison Ford, Phoebe Waller-Bridge, Antonio ...","55,758"
The Witcher,The Witcher,https://m.media-amazon.com/images/S/sash/4Fyxw...,2019–,mystery,None,"Geralt of Rivia, a solitary monster hunter, st...",8.1,Henry Cavill,"[Freya Allan, Anya Chalotra, Eamon Farren]","528,964"
Secret Invasion,Secret Invasion,https://m.media-amazon.com/images/S/sash/4Fyxw...,2023,mystery,None,Fury and Talos try to stop the Skrulls who hav...,6.9,Samuel L. Jackson,"[Ben Mendelsohn, Olivia Colman, Emilia Clarke]","14,558"
The Flash,The Flash,https://m.media-amazon.com/images/S/sash/4Fyxw...,1990–1991,fantasy,None,"A police forensic scientist, Barry Allen, batt...",7.1,John Wesley Shipp,"[Amanda Pays, Alex Désert, Vito D'Ambrosio]","7,837"
Spider-Man: Across the Spider-Verse,Spider-Man: Across the Spider-Verse,https://m.media-amazon.com/images/S/sash/4Fyxw...,2023,family,PG,"Miles Morales catapults across the Multiverse,...",8.9,Joaquim Dos Santos,"[Kemp Powers, Justin K. Thompson, Shameik Moor...","175,046"
...,...,...,...,...,...,...,...,...,...,...
Sunday with Laura Kuenssberg,Sunday with Laura Kuenssberg,https://m.media-amazon.com/images/S/sash/4Fyxw...,2022–,talk-show,None,Sunday morning political interview and discuss...,None,Laura Kuenssberg,"[Michael Gove, Ed Davey, Nicholas Soames]",None
The #AskAbhijit Show,The #AskAbhijit Show,https://m.media-amazon.com/images/S/sash/4Fyxw...,2020–,talk-show,None,Abhijit Chavda answers live questions from a g...,9.3,Abhijit Chavda,[],41
Saturday Fright at the Movies,Saturday Fright at the Movies,https://m.media-amazon.com/images/S/sash/4Fyxw...,1990–2000,talk-show,None,A host horror movie show with Danny Koker (aka...,8.1,Danny Koker,[],12
Actors Entertainment,Actors Entertainment,https://m.media-amazon.com/images/S/sash/4Fyxw...,2009–,talk-show,None,Test plot,8.3,Kurt Kelly,"[John Michael Ferrari, Pepper Jay, Yi Tian]",311


In [7]:
# Save DataFrame to Excel file
df.to_excel('imdb_dataset.xlsx', index=False)